# Social Network Analysis - Python Handson

In [ ]:
import numpy as np
from networkx import nx
import pandas as pd
import matplotlib.pyplot as plt

## The Data

The data set is provided by Der Standard, one of the top Austrian newspapers.
In the online Standard people can post comments below articles and up/down vote comments.
The data set used in this handson and further in the project part of the course will consider a sample of those articles, comments, and votes. 

In [ ]:
df = pd.read_csv('Sample_Articles_Int_081116_091116_Postings.csv', sep=';')

In [ ]:
df.head()

There are different entities in the data set: 
* Users - identified by *ID_CommunityIdentity* (or *UserCommunityName*)
* Postings - identified by *ID_Posting*
* Articles - identified by *ID_Article*

Thus, there are different possibilities to build networks based on the posting data. 
We will concentrate now on the ***reply-to-network***. 


In [ ]:
votes = pd.read_csv('Sample_Articles_Int_081116_091116_Votes.csv', sep=';')
votes.head()

## Reply-To-Network

The two fundamental components of a network are *nodes* and *edges*. 
In the anticipated reply-to-network nodes are the users (i.e., *ID_CommunityIdentity*). 
Edges between two nodes (i.e., users) are build if one user replys to a posting of another users. 

In [ ]:
df[["ID_CommunityIdentity", "ID_Posting", "ID_Posting_Parent"]].head(15)

A line in the table above shows that a user (i.e., *ID_CommunityIdentiy*) posted a comment. Every post has its own uniqe identifier (i.e., *ID_Posting*). If a user replys to a previous posting then the posting they are targeting is identified by *ID_Posting_Parent*. *NaN* shows that the posted comment is located in the root (i.e., it's not targeted towards any other comment). 

We want to bring the structure above into following format: 
* source, i.e., the replying user
* target, i.e., the targeted user
* weight, i.e., how often the source replied to the target

In other words, we are aiming for a *weighted edge-list*.

### Edges

In [ ]:
edgeList = [
    [post.ID_CommunityIdentity, next(iter(df[df.ID_Posting == post.ID_Posting_Parent].ID_CommunityIdentity))] 
    for idx, post in df.iterrows()
    if ~np.isnan(post.ID_Posting_Parent)]

In [ ]:
edgeList[:10]

In [ ]:
weightedEdgeList = [(edge[0],edge[1],edgeList.count(edge)) for edge in edgeList]
weightedEdgeList = list(set(weightedEdgeList))

In [ ]:
weightedEdgeList[:10]

In [ ]:
edges = pd.DataFrame(weightedEdgeList, columns=['source','target','weight'])

In [ ]:
edges.head()

In [ ]:
edges.to_csv("reply_to_edges.csv", index=False)

### Graph

We use the *networkx* library.
Since we build a *reply-to-network* we have *source* nodes and *target* nodes. 
Thus, the network is directed.
Therefore, we use *nx.Digraph()*

In [ ]:
G = nx.from_pandas_edgelist(edges, 
                            source='source', 
                            target='target', 
                            edge_attr = 'weight',
                            create_using=nx.DiGraph())

In [ ]:
fig = plt.figure(figsize=(50,50))
nx.draw_spring(G)
plt.show()

### Basic Stats

In [ ]:
print(nx.info(G))

In [ ]:
# number of edges with weight 1
len(edges[edges.weight == 1])

In [ ]:
# max weight of edges
edges.weight.max()

In [ ]:
# average weight 
edges.weight.mean()

### Network density and path lengths

In [ ]:
# network density
nx.density(G)

In [ ]:
# Average distance (i.e. average shortest path length)
nx.average_shortest_path_length(G)

The method average_shortest_path_length throws an exception if the underlying Graph is disconnected. Thus, one can calculate the average of all finite distances (i.e., existing shortest pathes) nx.single_source_shortest_path_length(G, N) delivers the length of all shortest pathes beginning from node N. Furthermore, the first shortest path is always the distance to itself (i.e., zero), which as to be filtered later on.

In [ ]:
# compute all distances
distances = [list(nx.single_source_shortest_path_length(G,N).values()) for N in G.nodes]
# Flatten the distances list! Currently list of lists of single node distances
# and filter out the unnecessary zeroes
distances = [distance for single_distances in distances for distance in single_distances if distance > 0]

In [ ]:
# average
np.mean(distances)

To consider the weight one can use e.g. nx.single_source_dijkstra_path_length() But watch out, what does weight in our case mean?

In [ ]:
# Diameter (i.e, longest shortest path)
np.max(distances)

### Connected components

G.subgraph(c) for c in nx.weakly_connected_components(G) delivers a Generator,which can be used to iterate over all weakly connected compontents (deliverd as a subgraph for further analysis)

In [ ]:
wccs = [c for c in (G.subgraph(c) for c in nx.weakly_connected_components(G))]

In [ ]:
# number of wccs
len(wccs)

In [ ]:
# number of wccs
nx.number_weakly_connected_components(G)

Sizes of the wccs:

nx.number_of_nodes() delivers the number of nodes of a graph. This can be done for all weakly connected components wcc in the weakly connected component list. Furthermore, with set() one can get the uniqe values.

In [ ]:
set([nx.number_of_nodes(wcc) for wcc in wccs])

If not the uniqe values are in focus, but the for example how often a wcc with n Nodes appear, one can use Counter().most_common() as follwing

In [ ]:
from collections import Counter
Counter([nx.number_of_nodes(wcc) for wcc in wccs]).most_common()

Strongly connected components:

In [ ]:
sccs = [c for c in (G.subgraph(c) for c in nx.strongly_connected_components(G))]
Counter([nx.number_of_nodes(scc) for scc in sccs]).most_common()

### Clustering Coefficients

**Local**

nx.clustering(G) returns back a dictionary with clustering coefficients of each node.
with the combination of sorted() and itemgetter() one can get a sorted list of (ID,clustering coeff.) tuples.

In [ ]:
from operator import itemgetter
sorted(nx.clustering(G).items(), key=itemgetter(1), reverse=True)[:5]

"*the clustering coefficient quantifies how close the neighbours of i are to being a clique.*" (lecture slides) i.e., how concentrated the neighbours of a nodes is.

**Global**

The global clustering coefficient can have alternative definitions:

1) as the average of the local clustering coefficients

In [ ]:
nx.average_clustering(G)

Note, that there might be differences if you use other tools (e.g., Gephi, Igraph, etc.).
So, why does networkx delivers a different average clustering coefficiet?
In order to find an answer, take a look at the nx.clustering() documentation (since nx.average_clustering is just averaging over the individual values). https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.cluster.clustering.html
It says, that clustering coefficients for nodes with degrees lower than 2 is set to ZERO.

Thus, there is no right or wrong way of implementation, but you have to be aware what you are using.

2) as the ratio of triangles and connected triples

In [ ]:
nx.transitivity(G)

### Centrality Indices

**In-Degree**

[nx.in_degree_centrality(G)](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.in_degree_centrality.html#networkx.algorithms.centrality.in_degree_centrality) delivers in-degree-centrality of each node in a graph G.
Note, that the centralities are normalized.


With a combination of sorted() and itemgetter() one can again get a sorted list of (Node, centrality) tuples.
Where one can just take the first 5 for reporting.
Note, reverse=True means in decreasing order



In [ ]:
sorted(nx.in_degree_centrality(G).items(), key=itemgetter(1), reverse=True)[:5]

User *588542* is replied the most.

**Out-Degree**

In [ ]:
sorted(nx.out_degree_centrality(G).items(), key=itemgetter(1), reverse=True)[:5]

User *588542* also replies the most

**Eigenvector-Centrality**

Eigenvector centrality computes the centrality for a node based on the centrality of its neighbors. The eigenvector centrality for node i is the i-th element of the vector 𝑥 defined by the equation

𝐴𝑥=𝜆𝑥

where 𝐴 is the adjacency matrix of the graph G with eigenvalue 𝜆. By virtue of the Perron–Frobenius theorem, there is a unique solution 𝑥, all of whose entries are positive, if 𝜆 is the largest eigenvalue of the adjacency matrix 𝐴
A.

In [ ]:
sorted(nx.eigenvector_centrality(G).items(), key=itemgetter(1), reverse=True)[:5]

**In-closeness centrality**

Closeness centrality of a node u is the reciprocal of the average shortest path distance to u over all n-1 reachable nodes.

𝐶(𝑢)= (𝑛−1)/∑𝑑(𝑣,𝑢)

where d(v, u) is the shortest-path distance between v and u, and n is the number of nodes that can reach u. Notice that the closeness distance function computes the incoming distance to u for directed graphs. To use outward distance, act on G.reverse().

In [ ]:
sorted(nx.closeness_centrality(G).items(), key=itemgetter(1), reverse=True)[:5]

**Out-closeness centrality**

In [ ]:
sorted(nx.closeness_centrality(G.reverse()).items(), key=itemgetter(1), reverse=True)[:5]

**Betweeness centrality**

Betweenness centrality of a node 𝑣 is the sum of the fraction of all-pairs shortest paths that pass through 𝑣

𝑐𝐵(𝑣)=∑𝜎(𝑠,𝑡|𝑣)/𝜎(𝑠,𝑡)

where, 𝜎(𝑠,𝑡) is the number of shortest (𝑠,𝑡)-paths, and 𝜎(𝑠,𝑡|𝑣) is the number of those paths passing through some node 𝑣 other than 𝑠,𝑡 .


Using k=100 nodes to estimate the betweeness centrality.

In [ ]:
sorted(nx.betweenness_centrality(G, k=100).items(), key=itemgetter(1), reverse=True)[:5]

### Link Analysis

**Hubs and Authorities**

In [ ]:
hubs_auth = nx.hits(G)

In [ ]:
# hub scores
sorted(hubs_auth[0].items(), key=itemgetter(1), reverse=True)[:5]

In [ ]:
# authority scores
sorted(hubs_auth[1].items(), key=itemgetter(1), reverse=True)[:5]

**Page Rank**

In [ ]:
sorted(nx.pagerank(G).items(), key=itemgetter(1), reverse=True)[:5]